In [57]:
from typing import TypedDict

class AgentState(TypedDict):
    user_query: str
    plan: list[str]
    current_step: int
    working_memory: dict
    invoked_agents: list[str]
    final_answer: str


In [58]:
AGENT_REGISTRY = {}

In [59]:
from langchain_groq import ChatGroq

rag_llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0.0
)


In [60]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings  # or Groq embeddings if available
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("C:/Users/SriHarsha/Desktop/POC/Q2. LG_Multi_model/home_loan.pdf")
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
# Create vector store
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)



C:\Users\SriHarsha\AppData\Local\Temp\ipykernel_15632\4065827976.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
def rag_agent(state: AgentState) -> AgentState:
    query = state["user_query"]

    # Retrieve relevant context from vector DB
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join(d.page_content for d in docs)

    prompt = f"""
    Extract ONLY factual information needed from the document.

    Context:
    {context}

    Question:
    {query}

    Do NOT calculate. Just extract facts.
    """

    extracted_info = rag_llm.invoke(prompt).content

    state["working_memory"]["rag_output"] = extracted_info
    return state


In [62]:
AGENT_REGISTRY["rag_agent"] = rag_agent

In [63]:
from langchain_openai import ChatOpenAI
import os

math_llm = ChatOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"],
    model="meta/llama3-8b-instruct",
    temperature=0
)


In [75]:
def math_agent(state: AgentState) -> AgentState:
    facts = state["working_memory"].get("rag_output", "")

    prompt = f"""
    Perform calculations based on the information below.
    Return ONLY the final numeric result.

    Information:
    {facts}
    """

    result = math_llm.invoke(prompt).content
    state["working_memory"]["math_result"] = result
    return state


In [65]:
AGENT_REGISTRY["math_agent"] = math_agent

In [66]:
from langchain_google_genai import ChatGoogleGenerativeAI

news_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [67]:
# def news_agent(state: AgentState) -> AgentState:
#     prompt = f"""
#     Provide the latest news updates on the following topic:

#     Topic: {state['user_query']}

#     Summarize key points concisely.
#     """
#     news = news_llm.invoke(prompt).content
#     return {
#         **state,
#         "response": news,
#         "iteration": state["iteration"] + 1
#     }


def news_agent(state: AgentState) -> AgentState:
    query = state["user_query"]

    prompt = f"""
    Provide the latest factual news related to:
    "{query}"

    Keep it concise and factual.
    """

    news = news_llm.invoke(prompt).content
    state["working_memory"]["news"] = news
    return state


In [68]:
AGENT_REGISTRY["news_agent"] = news_agent

In [69]:
planner_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [70]:
def supervisor_planner(state: AgentState) -> AgentState:
    """
    Uses an LLM to decide which agents should run and in what order.
    """
    prompt = f"""
    You are an AI planner.

    User query:
    "{state['user_query']}"

    Decide which agents are required and in what order.

    Available agents:
    - rag_agent (document extraction)
    - math_agent (calculations)
    - news_agent (latest news)

    Output ONLY a Python list, example:
    ["rag_agent", "math_agent"]
    """

    plan_text = planner_llm.invoke(prompt).content.strip()
    plan = eval(plan_text)  # safe if LLM constrained

    state["plan"] = plan
    state["current_step"] = 0
    state["working_memory"] = {}
    state["final_answer"] = ""
    state["invoked_agents"] = []

    return state


In [71]:
def controller(state: AgentState) -> AgentState:
    # ---- END CONDITION ----
    if state["current_step"] == len(state["plan"]):
        state["next"] = "end"
        return state

    if state["current_step"] > len(state["plan"]):
        state["next"] = "end"
        return state

    # ---- Execute agent ----
    agent_name = state["plan"][state["current_step"]]

    if len(state["invoked_agents"]) <= state["current_step"]:
        state["invoked_agents"].append(agent_name)

    agent_fn = AGENT_REGISTRY[agent_name]
    state = agent_fn(state)

    state["current_step"] += 1
    state["next"] = "controller"
    return state


In [72]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("supervisor", supervisor_planner)
graph.add_node("controller", controller)

graph.set_entry_point("supervisor")

graph.add_edge("supervisor", "controller")

graph.add_conditional_edges(
    "controller",
    lambda state: state["next"],
    {
        "controller": "controller",
        "end": END
    }
)

multi_agent_app = graph.compile()


In [74]:
user_question = input("Ask your question: ")

initial_state: AgentState = {
    "user_query": user_question,
    "plan": [],
    "current_step": 0,
    "working_memory": {},
    "invoked_agents": [],
    "next": "supervisor"
}

result = multi_agent_app.invoke(initial_state)

print("\n==============================")
print("AGENTS INVOKED:")
print(" → ".join(result["invoked_agents"]))

print("\n==============================")
print("AGENT OUTPUTS:")
for agent, output in result["working_memory"].items():
    print(f"\n{agent.upper()}:\n{output}")
# what is the LTV ratiofor home loan upto 30 lakh rupees and what it 11*883?



AGENTS INVOKED:
rag_agent → math_agent

AGENT OUTPUTS:

RAG_OUTPUT:
- The Loan‑to‑Value (LTV) ratio for a home loan up to Rs. 30 lacs is **90 %**.  
- The expression **11 × 883** represents the multiplication of 11 by 883.

MATH_RESULT:
Based on the information, I will perform the calculation:

11 × 883 = 9723

The final numeric result is: **9723**
